247.019

vns\
Selected Athletes: ['florian langenegger', 'henji mboyo', 'christian baumann', 'noe seifert']
Total Score: 250.19499999999996

hh
Best Selection: ['noe seifert' 'moreno kratter' 'henji mboyo' 'florian langenegger']
Best Score: 248.45299999999997

ts
['henji mboyo' 'noe seifert' 'christian baumann' 'florian langenegger']
Total Score:
250.19499999999996

In [23]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')


In [6]:
df=df[df['Nation']=='Switzerland']


In [7]:
df=df.drop(columns=['Rank', 'Nation', 'round', 'year', 'AA'])

In [8]:
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values
})


In [9]:
df = df_max_scores.copy()

In [10]:
import pandas as pd
import numpy as np
from itertools import combinations
from pulp import LpProblem, LpVariable, lpSum, LpMaximize


vns\
Selected Athletes: ['florian langenegger', 'henji mboyo', 'christian baumann', 'noe seifert']
Total Score: 250.19499999999996



In [11]:
# VNS Algorithm with Iteration through random_state values
def variable_neighborhood_search(df):
    apparatus_order = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']  

    def calculate_total_score(athletes_df):
        # Sum the top 3 scores for each apparatus
        total_score = athletes_df[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].apply(lambda x: x.nlargest(3).sum()).sum()
        return total_score

    best_athletes = None
    best_score = float('-inf')
    best_seed = None

    for seed in range(6000): 
        # Randomly select 4 athletes with the current seed
        selected_athletes = df.sample(4, random_state=seed)

        # Check if each apparatus is competed on 3 times
        if all(selected_athletes[apparatus].count() >= 3 for apparatus in apparatus_order):
            # Calculate the total score for the current solution
            current_score = calculate_total_score(selected_athletes)

            # Compare with the current best solution
            if current_score > best_score:
                best_athletes = selected_athletes['Athlete'].tolist()
                best_score = current_score
                best_seed = seed

    return best_athletes, best_score, best_seed

# Run the VNS algorithm with different random_state values
best_athletes, total_score, best_seed = variable_neighborhood_search(df)

print("Selected Athletes:", best_athletes)
print("Total Score:", total_score)
print("Best Seed:", best_seed)


Selected Athletes: ['florian langenegger', 'henji mboyo', 'christian baumann', 'noe seifert']
Total Score: 250.19499999999996
Best Seed: 15


hh
Best Selection: ['noe seifert' 'moreno kratter' 'henji mboyo' 'florian langenegger']
Best Score: 248.45299999999997



In [17]:
df = df_max_scores.copy()

In [18]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['noe seifert' 'moreno kratter' 'henji mboyo' 'florian langenegger']
Best Score: 248.45299999999997


ts
['henji mboyo' 'noe seifert' 'christian baumann' 'florian langenegger']
Total Score:
250.19499999999996

In [15]:
df = df_max_scores.copy()

In [16]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score



# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 249.927
Iteration 2, Best Score: 250.19499999999996
Iteration 3, Best Score: 250.19499999999996
Iteration 4, Best Score: 250.19499999999996
Iteration 5, Best Score: 250.19499999999996
Iteration 6, Best Score: 250.19499999999996
Iteration 7, Best Score: 250.19499999999996
Iteration 8, Best Score: 250.19499999999996
Iteration 9, Best Score: 250.19499999999996
Iteration 10, Best Score: 250.19499999999996
Iteration 11, Best Score: 250.19499999999996
Iteration 12, Best Score: 250.19499999999996
Iteration 13, Best Score: 250.19499999999996
Iteration 14, Best Score: 250.19499999999996
Iteration 15, Best Score: 250.19499999999996
Iteration 16, Best Score: 250.19499999999996
Iteration 17, Best Score: 250.19499999999996
Iteration 18, Best Score: 250.19499999999996
Iteration 19, Best Score: 250.19499999999996
Iteration 20, Best Score: 250.19499999999996
Iteration 21, Best Score: 250.19499999999996
Iteration 22, Best Score: 250.19499999999996
Iteration 23, Best Score: 250.

In [20]:
data = {
    'Method': ['vns', 'hh', 'ts'],
    'Selected Athletes': [
        ['florian langenegger', 'henji mboyo', 'christian baumann', 'noe seifert'],
        ['noe seifert', 'moreno kratter', 'henji mboyo', 'florian langenegger'],
        ['henji mboyo', 'noe seifert', 'christian baumann', 'florian langenegger']
    ],
    'Total Score': [250.195, 248.453, 250.195]
}

df = pd.DataFrame(data)

In [21]:
df

,Method,Selected Athletes,Total Score
0,vns,"[florian langenegger, henji mboyo, christian b...",250.195
1,hh,"[noe seifert, moreno kratter, henji mboyo, flo...",248.453
2,ts,"[henji mboyo, noe seifert, christian baumann, ...",250.195


In [22]:
# df.to_csv('ch.csv', index=False)

In [29]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')
df=df[df['Nation']=='Switzerland']
df=df.drop(columns=['Rank', 'Nation', 'round', 'year'])
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values,
    'AA': df.loc[idx_max['AA'], 'AA'].values

})


In [30]:
df=df_max_scores.copy()

In [31]:
df=df.sort_values(by='AA', ascending=False)

In [32]:
df

,Athlete,VT,PH,SR,FX,PB,HB,AA
3,henji mboyo,14.058,13.700,13.400,13.700,13.933,13.933,82.691
6,noe seifert,14.100,13.566,13.366,14.200,14.766,13.733,82.464
2,florian langenegger,14.300,13.466,12.933,14.200,13.866,13.233,81.864
0,andrin frey,13.200,12.366,12.566,13.966,12.300,10.866,73.431
5,moreno kratter,14.000,0.000,12.600,13.833,13.833,12.133,66.399
1,christian baumann,0.000,13.500,13.666,0.000,14.841,13.233,54.465
7,taha serhani,14.366,12.533,0.000,11.700,14.300,13.700,54.032
4,luca giubellini,14.366,11.433,0.000,13.866,0.000,0.000,39.665


In [33]:
82.691+82.464+81.864

247.019